In [14]:
import numpy as np
from collections import defaultdict
import gtnlplib.preproc
import gtnlplib.preproc_metrics

import gtnlplib.clf_base
import gtnlplib.wordlist
import gtnlplib.naivebayes
import gtnlplib.perceptron
import gtnlplib.avg_perceptron
import gtnlplib.logreg

import gtnlplib.scorer
import gtnlplib.constants
import gtnlplib.analysis
import gtnlplib.weight_ensemble
# this enables you to create inline plots in the notebook 
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [46]:
### TRAINKEY, DEVKEY and TESTKEY are defined in the gtnlplib.constants module
reload(gtnlplib.preproc)
gtnlplib.preproc.docsToBOWs(gtnlplib.constants.TRAINKEY)
gtnlplib.preproc.docsToBOWs(gtnlplib.constants.DEVKEY)
## uncomment once you have the test data
gtnlplib.preproc.docsToBOWs(gtnlplib.constants.TESTKEY)

In [47]:
reload(gtnlplib.preproc)
ac_train = gtnlplib.preproc.getAllCounts(gtnlplib.preproc.dataIterator(gtnlplib.constants.TRAINKEY))
ac_dev = gtnlplib.preproc.getAllCounts(gtnlplib.preproc.dataIterator(gtnlplib.constants.DEVKEY))
print "number of word types",len(ac_train.keys())-1

number of word types 113014


# 3. Word list classification #
(_setting weights - 2 pts, Deliverable 3 - 1 pt. Total 3 pts_)

In [3]:
poswords, negwords = gtnlplib.wordlist.loadSentimentWords (gtnlplib.constants.SENTIMENT_FILE)

In [4]:
reload(gtnlplib.wordlist)
weights_wlc = gtnlplib.wordlist.learnWLCWeights (poswords, negwords)
outfile = 'word_list.txt'
mat = gtnlplib.clf_base.evalClassifier(weights_wlc,outfile, gtnlplib.constants.DEVKEY)
print gtnlplib.scorer.printScoreMessage(mat)

3 classes in key: set(['NEG', 'NEU', 'POS'])
3 classes in response: set(['NEG', 'NEU', 'POS'])
confusion matrix
key\response:	NEG	NEU	POS
NEG		36	14	61	
NEU		22	14	99	
POS		9	13	126	
----------------
accuracy: 0.4467 = 176/394

None


# 4. Naive Bayes #

In [15]:
counts, class_counts,allkeys = gtnlplib.preproc.getCountsAndKeys(gtnlplib.constants.TRAINKEY)
reload(gtnlplib.naivebayes)
weights_nb = gtnlplib.naivebayes.learnNBWeights (counts, class_counts, allkeys, alpha=1.3)

In [16]:
outfile = 'nb.txt'
mat = gtnlplib.clf_base.evalClassifier(weights_nb,outfile, gtnlplib.constants.DEVKEY)
print gtnlplib.scorer.printScoreMessage(mat)

3 classes in key: set(['NEG', 'NEU', 'POS'])
3 classes in response: set(['NEG', 'NEU', 'POS'])
confusion matrix
key\response:	NEG	NEU	POS
NEG		15	85	11	
NEU		12	97	26	
POS		8	46	94	
----------------
accuracy: 0.5228 = 206/394

None


In [7]:
alphas = [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 1.3, 1.5, 1.8, 2, 5, 10] #your choice!

In [8]:
weights_nb_alphas, tr_accs, dv_accs = gtnlplib.naivebayes.regularization_using_grid_search (alphas,counts, class_counts, allkeys)

In [9]:
for i,alpha in enumerate (alphas):
    print alpha, tr_accs[i], dv_accs[i]

0.0001 0.987820512821 0.467005076142
0.001 0.986538461538 0.472081218274
0.01 0.985897435897 0.507614213198
0.1 0.982692307692 0.517766497462
0.5 0.962179487179 0.515228426396
1 0.94358974359 0.522842639594
1.3 0.93141025641 0.522842639594
1.5 0.924358974359 0.522842639594
1.8 0.904487179487 0.522842639594
2 0.896153846154 0.520304568528
5 0.712820512821 0.464467005076
10 0.605128205128 0.444162436548


# 6. Perceptron #
(_ 6 points total _)

Implement a perceptron classifier. Using the feature-function
representation, include features for each word-class pair, and also an
** offset ** feature for each class. Given a set of word counts $\vec{x}_i$,
a true label $y_i$, and a guessed label $\hat{y}$, your update will be
\begin{align*}
\hat{y} & \leftarrow \text{argmax}_y \vec{\theta}' f(\vec{x}_i,y)\\
\vec{\theta} & \leftarrow \vec{\theta} + f(\vec{x}_i, y_i) - f(\vec{x}_i, \hat{y}).
\end{align*}

Please write this yourself -- do not use any libraries, and try not to look
at other code online.

**Sanity check** If you are not careful, learning can be slow. 
You may need to think a little about how to do this update efficiently. 

- On my laptop, I can make 10 passes on the training data in roughly 30 seconds, including evaluating the accuracy on the dev and training sets. 
- You can use the ```%%timeit``` cell magic to compute statistics like this.
- Your code doesn't have to be as fast as mine, but it needs to be written intelligently, and it needs to be fast enough for you to debug it properly.
- The ```%%prun``` cell magic is also useful for diagnosing speed

In [48]:
all_tr_insts,all_dev_insts= gtnlplib.preproc.loadInstances(gtnlplib.constants.TRAINKEY, gtnlplib.constants.DEVKEY)

In [22]:
reload(gtnlplib.perceptron)
outfile = "perc.txt"
w_perc,tr_acc_perc,dv_acc_perc = gtnlplib.perceptron.trainPerceptron(27, all_tr_insts,gtnlplib.constants.ALL_LABELS, outfile, gtnlplib.constants.DEVKEY)

0 dev:  0.428934010152 train:  0.400641025641
1 dev:  0.418781725888 train:  0.533974358974
2 dev:  0.5 train:  0.616666666667
3 dev:  0.403553299492 train:  0.662179487179
4 dev:  0.505076142132 train:  0.657692307692
5 dev:  0.560913705584 train:  0.755128205128
6 dev:  0.527918781726 train:  0.798076923077
7 dev:  0.477157360406 train:  0.839102564103
8 dev:  0.46192893401 train:  0.790384615385
9 dev:  0.52538071066 train:  0.836538461538
10 dev:  0.535532994924 train:  0.882692307692
11 dev:  0.530456852792 train:  0.898717948718
12 dev:  0.52538071066 train:  0.917307692308
13 dev:  0.555837563452 train:  0.90641025641
14 dev:  0.545685279188 train:  0.933333333333
15 dev:  0.520304568528 train:  0.955769230769
16 dev:  0.540609137056 train:  0.967948717949
17 dev:  0.560913705584 train:  0.967307692308
18 dev:  0.530456852792 train:  0.958974358974
19 dev:  0.482233502538 train:  0.95641025641
20 dev:  0.472081218274 train:  0.971794871795
21 dev:  0.530456852792 train:  0.97179

# 7. Averaged Perceptron #

In [12]:
reload(gtnlplib.avg_perceptron)
# again, this takes roughly 30 seconds for me
outfile = "ap.txt"
w_ap,tr_acc_ap,dv_acc_ap = gtnlplib.avg_perceptron.trainAvgPerceptron(10,all_tr_insts,gtnlplib.constants.ALL_LABELS, outfile,gtnlplib.constants.DEVKEY)

0 dev:  0.484771573604 train:  0.400641025641
1 dev:  0.517766497462 train:  0.533974358974
2 dev:  0.530456852792 train:  0.616666666667
3 dev:  0.55076142132 train:  0.662179487179
4 dev:  0.532994923858 train:  0.657692307692
5 dev:  0.555837563452 train:  0.755128205128
6 dev:  0.568527918782 train:  0.798076923077
7 dev:  0.578680203046 train:  0.839102564103
8 dev:  0.573604060914 train:  0.790384615385
9 dev:  0.573604060914 train:  0.836538461538


# 8. Logistic regression #

Now you will complete an implementation of logistic regression.
We've provided a lot of scaffolding code, you just need to fill in some key parts.

In [51]:
reload(gtnlplib.logreg)
outfile = "sgd.txt"
w_sgd,tr_acc_sgd,dv_acc_sgd = gtnlplib.logreg.trainLRbySGD(200,all_tr_insts, outfile, gtnlplib.constants.DEVKEY, regularizer=1e-1)

0 dev: 0.492385786802 train: 0.424631173829
1 dev: 0.502538071066 train: 0.559332905709
2 dev: 0.505076142132 train: 0.594611930725
3 dev: 0.507614213198 train: 0.60936497755
4 dev: 0.507614213198 train: 0.62604233483
5 dev: 0.51269035533 train: 0.629249518922
6 dev: 0.51269035533 train: 0.64271969211
7 dev: 0.517766497462 train: 0.658114175754
8 dev: 0.517766497462 train: 0.669660038486
9 dev: 0.517766497462 train: 0.679923027582
10 dev: 0.515228426396 train: 0.689544579859
11 dev: 0.51269035533 train: 0.698524695318
12 dev: 0.51269035533 train: 0.712636305324
13 dev: 0.520304568528 train: 0.71840923669
14 dev: 0.522842639594 train: 0.730596536241
15 dev: 0.522842639594 train: 0.737652341244
16 dev: 0.522842639594 train: 0.744708146248
17 dev: 0.52538071066 train: 0.753688261706
18 dev: 0.522842639594 train: 0.754329698525
19 dev: 0.522842639594 train: 0.762668377165
20 dev: 0.52538071066 train: 0.76908274535
21 dev: 0.52538071066 train: 0.773572803079
22 dev: 0.530456852792 train: 0.

# 10. Bakeoff! #

48 hours before the assignment is due, I will send you unlabeled test
data. Your job is to produce a response file, and submit it to our Kaggle
bakeoff ([link here](https://inclass.kaggle.com/c/gt-book-review-sentiment-analysis)).
The Kaggle contest compares your classifier's results on the dev data to generate a
class-visible leaderboard, and compares your classifier's results on the unlabeled
test data for the bakeoff. You can use the dev data results as a sanity
check, to make sure you submit the correct file.

I'll present the results in class and give the best scorers a chance to explain
what they did.

** Deliverable 10 ** (3 points) Run your best system from any part of the
assignment on the test data using the `generateKaggleSubmission()` function. Submit
your response file to the class [Kaggle bakeoff](https://inclass.kaggle.com/c/gt-book-review-sentiment-analysis). Also submit your Kaggle response file to T-Square as 'lastname-firstname.response'. The top
scores will be announced in class.

In [43]:
reload(gtnlplib.weight_ensemble)
reload(gtnlplib.clf_base)
best_acc = 0.0
for w4 in np.arange(0.8, 10, 0.1):
    for w3 in np.arange(0.1, 10, 0.1):
                w1 = 2.4
                w2 = 1.2
                avg_weight = gtnlplib.weight_ensemble.ensemble([(weights_wlc, w1), (weights_nb, w2), (w_ap, w3), (w_sgd, w4)])
                yourBestWeights = avg_weight # Change this to your best model
                acc = gtnlplib.clf_base.generateKaggleSubmission(yourBestWeights, 'Dai-Hanjun.response')
                if acc > best_acc:
                    best_acc = acc
                    best_w1 = w1
                    best_w2 = w2
                    best_w3 = w3
                    best_w4 = w4
                    print best_acc, w1, w2, w3, w4

0.634517766497 2.4 1.2 0.1 0.8


KeyboardInterrupt: 